In [1]:
import torch
from torch import nn

torch.__version__

'1.13.1'

In [2]:
# Setup device agnostic code
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [7]:
import requests
import zipfile
from pathlib import Path

# Setup path to a data folder
data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f'{image_path} directory alaready exists... skipping download')
else:
    print(f'{image_path} does not exist, creating one...')
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak and sushi data
with open(data_path / 'pizza_steak_sushi.zip','wb') as f:
    request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
    print('Donwloading pizza, steak, sushi data...')
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / 'pizza_steak_sushi.zip','r') as zip_ref:
    print('Unzipping piza, steak, sushi data...')
    zip_ref.extractall(image_path)

data/pizza_steak_sushi directory alaready exists... skipping download
Donwloading pizza, steak, sushi data...
Unzipping piza, steak, sushi data...


In [8]:
import os 
def walk_through_dir(dir_path):
    '''Walks through dir_path directory returning it's contents.'''
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f'there are {len(dirnames)} directories and {len(filenames)} images in "{dirpath}".')

In [9]:
walk_through_dir(image_path)

there are 2 directories and 0 images in "data/pizza_steak_sushi".
there are 3 directories and 0 images in "data/pizza_steak_sushi/test".
there are 0 directories and 19 images in "data/pizza_steak_sushi/test/steak".
there are 0 directories and 31 images in "data/pizza_steak_sushi/test/sushi".
there are 0 directories and 25 images in "data/pizza_steak_sushi/test/pizza".
there are 3 directories and 0 images in "data/pizza_steak_sushi/train".
there are 0 directories and 75 images in "data/pizza_steak_sushi/train/steak".
there are 0 directories and 72 images in "data/pizza_steak_sushi/train/sushi".
there are 0 directories and 78 images in "data/pizza_steak_sushi/train/pizza".
